In [1]:
import datetime

start = datetime.datetime.now()
start

datetime.datetime(2021, 11, 5, 14, 36, 46, 456714)

In [2]:
import pandas as pd
import sqlite3
import json
from tqdm.auto import tqdm

In [3]:
db = sqlite3.connect("datasets.db")
cur = db.cursor()

In [4]:
keys = [
    "id",
    "smart_collapse_field",
    "title",
    "rubric_id",

    "author_id",
    "author_fullName",
    
    "available",
    "year_value",
]

In [5]:
! wc -l ../data/books_full.jsonlines

2181593 ../data/books_full.jsonlines


In [6]:
data = []

k = 0
for line in tqdm(open("../data/books_full.jsonlines"), total=2181593):
    try:
        row = json.loads(line)
        res = []
        for key in keys:
            res.append(row.get(key))
        data.append(res)
        k += 1
    except:
        print("-")

  0%|          | 0/2181593 [00:00<?, ?it/s]

-


In [7]:
len(data)

2181592

In [8]:
df = pd.DataFrame(data, columns=keys)

In [9]:
df.head(2)

,id,smart_collapse_field,title,rubric_id,author_id,author_fullName,available,year_value
0,397272,0c27869e847e67cedba21d2767b6434c,Великие правители,13.0,NaN,None,1,2005.0
1,397417,9f6e64326afbfa79f11d7c31d5efa5a5,Волею императрицы : Ист.романы,479.0,126874.0,Щепкина А. В.,1,1996.0


In [10]:
smart_collapse = {v: i for i, v in enumerate(df["smart_collapse_field"].unique())}

In [11]:
def toint(x):
    try:
        if type(x) == int:
            return x
        if type(x) == str and "." in x:
            return int(x.split(".")[0])
        return int(x)
    except:
        return -1

In [12]:
for key in ["rubric_id", "author_id", "year_value"]:
    df[key] = df[key].apply(toint)

In [13]:
df["smart_collapse_field"] = df["smart_collapse_field"].apply(smart_collapse.get)

In [14]:
df[["id", "smart_collapse_field"]].to_sql("smart_collapse_map", con=db, index=False, if_exists="replace")

In [15]:
rubrics = pd.read_csv("../data/rubrics.csv", usecols=["level1", "level2", "id"])
rubrics.columns = ["rubric0", "rubric1", "rubric_id"]

In [16]:
df = df.merge(rubrics, how="left")

In [17]:
rubrics = df.groupby(["smart_collapse_field", "author_id", "rubric0", "rubric1"], as_index=False).agg({"id": "count"})
rubrics.columns = ["smart_collapse_field", "author_id", "rubric0", "rubric1", "cnt"]

In [18]:
df = df.merge(rubrics, how="left").sort_values(by=["cnt", "rubric1"], ascending=False)

In [19]:
def tr_min(x):
    lst = [i for i in x if i != -1]
    if lst:
        return min(lst)
    else:
        return -1

def tr_max(x):
    lst = [i for i in x if i != -1]
    if lst:
        return max(lst)
    else:
        return -1

In [20]:
df["available"] = df.groupby("smart_collapse_field")["available"].transform(tr_max)

In [21]:
df["author_id"] = df.groupby("smart_collapse_field")["author_id"].transform(tr_min)

In [22]:
df = df.drop_duplicates("smart_collapse_field", keep="first")

In [23]:
for key in ["rubric0", "rubric1"]:
    df[key] = df[key].apply(toint)

In [24]:
df[["id", "smart_collapse_field", "title", "rubric0", "rubric1", "rubric_id", "author_id", "available", "year_value"]].to_sql("books", con=db, index=False)

In [25]:
df[["author_id", "author_fullName"]].drop_duplicates().to_sql("authors", con=db, index=False)

In [26]:
for key in ["rubric_id", "author_id", "year_value", "rubric0", "rubric1"]:
    cur.execute(f"UPDATE books SET {key} = NULL WHERE {key} = -1")
    db.commit()

In [27]:
rubrics = pd.read_csv("../data/rubrics.csv")
rubrics.columns = ["rubric0", "rubric1", "rubric_id", "name"]
rubrics.to_sql("rubrics", con=db, index=False)

In [5]:
PATH = "../data/"

In [6]:
data = []
for i in tqdm(range(1, 17)):
    df = pd.read_csv(PATH + f"circulaton_{i}.csv", encoding="cp1251", sep=";")
    df = df[["catalogueRecordID", "readerID", "startDate"]]
    df["startDate"] = df["startDate"].apply(lambda x: "-".join(reversed(x.split("."))))
    data.append(df)

  0%|          | 0/16 [00:00<?, ?it/s]

In [7]:
cur.execute("SELECT id, smart_collapse_field FROM books")
smart_collapse = dict(cur.fetchall())

In [8]:
df = pd.concat(data)

In [9]:
df["smart"] = df["catalogueRecordID"].apply(smart_collapse.get)

In [12]:
df = df.dropna(subset=["smart"]).sort_values(by="startDate").drop_duplicates(subset=["smart", "readerID"], keep="first")
df["smart"] = df["smart"].apply(toint)

In [13]:
df.to_sql("circulation_short", con=db, index=False, if_exists="replace")

In [14]:
cur.execute('CREATE INDEX "aidx" ON "authors" ("author_id");')
cur.execute('CREATE INDEX "bidx" ON "books" ("smart_collapse_field");')
cur.execute('CREATE INDEX "ridx" ON "rubrics" ("rubric_id");')
cur.execute('CREATE INDEX "cidx" ON "circulation_short" ("smart");')
cur.execute('CREATE INDEX "miidx" ON "smart_collapse_map" ("id");')
cur.execute('CREATE INDEX "mcidx" ON "smart_collapse_map" ("smart_collapse_field");')
db.commit()

In [36]:
print(datetime.datetime.now())
print(datetime.datetime.now() - start)

2021-11-05 14:12:13.540512
0:07:21.762813
